In [3]:
from tensorflow.keras.metrics import MeanIoU, Recall, Precision, Accuracy
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import SGD, Adam,schedules
from sklearn.metrics import confusion_matrix
import numpy as np




In [4]:
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
import random
from tensorflow.keras.utils import to_categorical
import cv2
import matplotlib.pyplot as plt
TRAIN_SPLIT = 0.85

DIR = './'
imgH = 32
imgW = 32

# file = open('data_index.txt', 'w')
count = 0

def load_image_data():
    imgDir = DIR + 'Normal/'
    imgSet1 = prepare_image_array(imgDir, imgW, imgH)
    m = imgSet1.shape[0]

    imgDir = DIR + 'OverExposed/'
    imgSet2 = prepare_image_array(imgDir, imgW, imgH)
    n = imgSet2.shape[0]

    imgDir = DIR + 'UnderExposed/'
    imgSet3 = prepare_image_array(imgDir, imgW, imgH)
    o = imgSet3.shape[0]

    # Put all image data into one array.
    imgSet = np.concatenate((imgSet1, imgSet2, imgSet3), axis=0)
    print(imgSet.shape)

    labelSet1 = np.zeros(m, dtype=np.uint8)
    labelSet2 = np.ones(n, dtype=np.uint8)
    label_o = np.ones(o, dtype=np.uint8)
    labelSet3 = np.add(label_o, label_o)
    labelSet = np.concatenate((labelSet1, labelSet2, labelSet3), axis=0)


    p = imgSet.shape[0]  # p = n + m + o

    # random.shuffle(indices)
    # for i in range(len(indices)):
    #     file.write(str(indices[i])+",")
    
    # file.close()
    f = open('data_index.txt', 'r')
    indices = []
    for x in f.read().split(','):
        if len(x) > 0:
            num = ''
            for j in x:
                if(j>='0' and j<='9'):
                    num +=j;
            if len(num)>0 and  int(num)<p:
                indices.append(int(num)) 
   
    # print(indices)
    imgSet = imgSet[indices]
    labelSet = labelSet[indices]
    print(labelSet[:5])
    print("Label set : ")
    labelSet = labelSet.reshape(p, 1)

#   one hot encoding

    labelSet = to_categorical(labelSet)

    # print(labelSet[:5])
    
    # print(imgSet[0])
    imgSet = imgSet.astype(np.float32)
    imgSet = imgSet/255.0

    r = int(p * TRAIN_SPLIT)
    trainX = imgSet[:r]
    trainY = labelSet[:r]
    testX = imgSet[r:]
    testY = labelSet[r:]

    return trainX, trainY, testX, testY



def prepare_image_array(imgDir, imgW, imgH):
    imgList = os.listdir(imgDir)
    # print(imgList)
    n = len(imgList)

    imgSet = []
    for i in range(n):
        imgPath = imgDir + imgList[i]
        if (os.path.exists(imgPath)):
            # print(imgPath)
            img = cv2.imread(imgPath)
            resizedImg = cv2.resize(img, (imgW, imgH))
            rgbImg = cv2.cvtColor(resizedImg, cv2.COLOR_BGR2RGB)
            imgSet.append(rgbImg)
        else:
            print("It is not a valid image path.")

    # print("total image "+str(len(imgSet)))
    imgSet = np.array(imgSet, dtype=np.uint8)
    # print("total shape "+str(imgSet.shape))

    return imgSet



In [5]:
train_x,train_Y, testInSet, testOutSet =load_image_data()


(8984, 32, 32, 3)
[0 2 1 2 0]
Label set : 


In [6]:
from sklearn.metrics import classification_report

def run_model(modelpath,testInSet, testOutSet):

    lr_schedule = schedules.ExponentialDecay(
        initial_learning_rate=0.01,
        decay_steps=10000,
        decay_rate=0.9)

    sgd = SGD(learning_rate=lr_schedule)


    model = load_model(modelpath, compile = True)
    model.compile(loss = 'mse', optimizer = sgd, metrics = ['accuracy', Recall(), Precision()])
    


#     y_test = np.argwhere(testOutSet)[:,1]
    # print(y_test)
#     indices = np.argwhere(y_test == 0).flatten()
#     print(indices.shape)
#     testInNormal = testInSet[indices]
#     testOutNormal = testOutSet[indices]



#     indices = np.argwhere(y_test == 1).flatten()
#     testInOver = testInSet[indices]
#     testOutOver = testOutSet[indices]
#     indices = np.argwhere(y_test == 2).flatten()
#     testInUnder = testInSet[indices]
#     testOutUnder = testOutSet[indices]



    '''	Test Model. '''
    lossT, accT, precisionT, recallT = model.evaluate(testInSet, testOutSet)
#     lossN, accN, precisionN, recallN = model.evaluate(testInNormal, testOutNormal)
#     lossO, accO, precisionO, recallO = model.evaluate(testInOver, testOutOver)
#     lossU, accU, precisionU, recallU = model.evaluate(testInUnder, testOutUnder)


    print("\n\nlossT = %s, accT = %s,  precisionT = %s, recallT = %s "%(lossT, accT, precisionT, recallT))
#     print("lossN = %s, accN = %s,  precisionN = %s, recallN = %s "%(lossN, accN, precisionN, recallN))
#     print("lossO = %s, accO = %s,  precisionO = %s, recallO = %s "%(lossO, accO, precisionO, recallO))
#     print("lossU = %s, accU = %s,  precisionU = %s, recallU = %s "%(lossU, accU, precisionU, recallU))

    predicted_output = []
    prediction = model.predict(testInSet)
    for i in prediction:
        predicted_output.append(np.argmax(i)) 

    actual_output = []
    for i in testOutSet:
        actual_output.append(np.argmax(i)) 
    con_mat = confusion_matrix(actual_output, predicted_output)
#     for i in con_mat:
#         print(i)
        
    cm =  classification_report(actual_output,predicted_output )
    print(cm)
    cm = cm.split(" ")
    count = 0
    cnt = 0
    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "
    acc = "Accuracy & \multicolumn{3}{r}{"
    mac = "Macro Avg "
    weight = "Weighted Avg "
    for i in cm:
        if len(i)==4:
            count = 0
            for j in i:
                if j=='.':
                    count =1
            if count ==1:
                if(cnt<3):
                    first_line +=" & %s"%i[2:]
                    first_line +='\%'
                if cnt>=3 and cnt<6:
                    second_line += " & %s"%i[2:]
                    second_line +='\%'
                if cnt>=6 and  cnt<9:
                    third_line += " & %s"%i[2:]
                    third_line +='\%'
                if cnt>=9 and cnt<10:
                    acc += " %s"%i[2:]
                    acc +='\%}'
                if cnt>=10 and cnt<13:
                    mac +=" & %s"%i[2:]
                    mac +='\%'
                if cnt>=13 and cnt<16:
                    weight += " & %s"%i[2:]
                    weight +='\%'
                cnt+=1

    print(first_line+'\\\\')
    print(second_line+'\\\\')
    print(third_line+'\\\\\\hline')
    print(acc+'\\\\')
    print(mac+'\\\\')
    print(weight+'\\\\\hline\hline\\thinlines')



    print("\n\n\n\n")


    first_line = "Normal "
    second_line = "Over Expsoed "
    third_line = "Under Expsoed "

    count = 0
    for i in con_mat:
        if count==0:
            first_line+= ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==1:
            second_line+=  ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        if count==2:
            third_line+=   ' & '+str(i[0])+ ' & '+str(i[1])+  ' & '+str(i[2])
        count+=1


    print(first_line+" \\\\\hline")
    print(second_line+" \\\\\hline")
    print(third_line+" \\\\\hline")

    
    return cm,con_mat

    

In [7]:
print("Non trainable")
DIR = './non-trainable/'
modelpath = DIR + 'VGG_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Non trainable
43/43 [==============================] - 3s 67ms/step - loss: 0.2225 - accuracy: 0.3093 - recall: 0.0000e+00 - precision: 0.0000e+00


lossT = 0.22250288724899292, accT = 0.3093471825122833,  precisionT = 0.0, recallT = 0.0 
              precision    recall  f1-score   support

           0       0.31      0.88      0.46       421
           1       0.36      0.01      0.02       452
           2       0.31      0.09      0.13       475

    accuracy                           0.31      1348
   macro avg       0.32      0.33      0.20      1348
weighted avg       0.32      0.31      0.20      1348

Normal  & 31\% & 88\% & 46\%\\
Over Expsoed  & 36\% & 01\% & 02\%\\
Under Expsoed  & 31\% & 09\% & 13\%\\\hline
Accuracy & \multicolumn{3}{r}{ 31\%}\\
Macro Avg  & 32\% & 33\% & 20\%\\
Weighted Avg  & 32\% & 31\% & 20\%\\\hline\hline\thinlines





Normal  & 371 & 6 & 44 \\\hline
Over Expsoed  & 399 & 5 & 48 \\\hline
Under Expsoed  & 431 & 3 & 41 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.31',
  '',
  '',
  '',
  '',
  '',
  '0.88',
  '',
  '',
  '',
  '',
  '',
  '0.46',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.36',
  '',
  '',
  '',
  '',
  '',
  '0.01',
  '',
  '',
  '',
  '',
  '',
  '0.02',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.31',
  '',
  '',
  '',
  '',
  '',
  '0.09',
  '',
  '',
  '',
  '',
  '',
  '0.13',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [6]:
print("trainable")
DIR = './trainable/'
modelpath = DIR + 'trainable-tf-Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

trainable
43/43 [==============================] - 3s 69ms/step - loss: 0.1157 - accuracy: 0.7493 - recall_1: 0.7389 - precision_1: 0.7626


lossT = 0.11571825295686722, accT = 0.7492581605911255,  precisionT = 0.7388724088668823, recallT = 0.7626339793205261 
Normal  & 61\% & 64\% & 62\%\\
Over Expsoed  & 77\% & 85\% & 81\%\\
Under Expsoed  & 87\% & 76\% & 81\%\\\hline
Accuracy & \multicolumn{3}{r}{ 75\%}\\
Macro Avg  & 75\% & 75\% & 75\%\\
Weighted Avg  & 76\% & 75\% & 75\%\\\hline\hline\thinlines





Normal  & 268 & 107 & 46 \\\hline
Over Expsoed  & 61 & 382 & 9 \\\hline
Under Expsoed  & 111 & 4 & 360 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.61',
  '',
  '',
  '',
  '',
  '',
  '0.64',
  '',
  '',
  '',
  '',
  '',
  '0.62',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.77',
  '',
  '',
  '',
  '',
  '',
  '0.85',
  '',
  '',
  '',
  '',
  '',
  '0.81',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.87',
  '',
  '',
  '',
  '',
  '',
  '0.76',
  '',
  '',
  '',
  '',
  '',
  '0.81',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [20]:
print("fully connected ")
DIR = './Fully-Connnected-model/'
modelpath = DIR + 'Fully_connected_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

fully connected 
43/43 [==============================] - 1s 4ms/step - loss: 0.0981 - accuracy: 0.8131 - recall_11: 0.8131 - precision_11: 0.8131


lossT = 0.09814359247684479, accT = 0.8130564093589783,  precisionT = 0.8130564093589783, recallT = 0.8130564093589783 
Normal  & 71\% & 76\% & 73\%\\
Over Expsoed  & 92\% & 75\% & 82\%\\
Under Expsoed  & 83\% & 93\% & 87\%\\\hline
Accuracy & \multicolumn{3}{r}{ 81\%}\\
Macro Avg  & 82\% & 81\% & 81\%\\
Weighted Avg  & 82\% & 81\% & 81\%\\\hline\hline\thinlines





Normal  & 319 & 28 & 74 \\\hline
Over Expsoed  & 96 & 337 & 19 \\\hline
Under Expsoed  & 34 & 1 & 440 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.71',
  '',
  '',
  '',
  '',
  '',
  '0.76',
  '',
  '',
  '',
  '',
  '',
  '0.73',
  '',
  '',
  '',
  '',
  '',
  '',
  '421\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.92',
  '',
  '',
  '',
  '',
  '',
  '0.75',
  '',
  '',
  '',
  '',
  '',
  '0.82',
  '',
  '',
  '',
  '',
  '',
  '',
  '452\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.83',
  '',
  '',
  '',
  '',
  '',
  '0.93',
  '',
  '',
  '',
  '',
  '',
  '0.87',
  '',
  '',
  '',
  '',
  '',
  '',
  '475\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

In [13]:
print("smalll")
DIR = './Small-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Small_CNN_Classifier.hdf5'

cm,con_mat = run_model(modelpath,testInSet, testOutSet)




smalll
43/43 [==============================] - 1s 4ms/step - loss: 0.1054 - accuracy: 0.7834 - recall_8: 0.7678 - precision_8: 0.7865


lossT = 0.10542751103639603, accT = 0.783382773399353,  precisionT = 0.7678041458129883, recallT = 0.7864741683006287 
Normal  & 67\% & 68\% & 68\%\\
Over Expsoed  & 84\% & 80\% & 82\%\\
Under Expsoed  & 83\% & 86\% & 84\%\\\hline
Accuracy & \multicolumn{3}{r}{ 78\%}\\
Macro Avg  & 78\% & 78\% & 78\%\\
Weighted Avg  & 78\% & 78\% & 78\%\\\hline\hline\thinlines





Normal  & 288 & 67 & 66 \\\hline
Over Expsoed  & 75 & 361 & 16 \\\hline
Under Expsoed  & 67 & 1 & 407 \\\hline


In [96]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)

Large-Size-Convolution-Maxpooling-model
42/42 [==============================] - 2s 28ms/step - loss: 0.2099 - accuracy: 0.5879 - recall_76: 0.0000e+00 - precision_76: 0.0000e+00


lossT = 0.20994505286216736, accT = 0.5878832936286926,  precisionT = 0.0, recallT = 0.0 
Normal  & 36\% & 28\% & 32\%\\
Over Expsoed  & 57\% & 94\% & 71\%\\
Under Expsoed  & 88\% & 53\% & 66\%\\\hline
Accuracy & \multicolumn{3}{r}{ 59\%}\\
Macro Avg  & 60\% & 58\% & 56\%\\
Weighted Avg  & 62\% & 59\% & 57\%\\\hline\hline\thinlines





Normal  & 117 & 269 & 31 \\\hline
Over Expsoed  & 25 & 419 & 4 \\\hline
Under Expsoed  & 181 & 41 & 250 \\\hline


(['',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'precision',
  '',
  '',
  '',
  'recall',
  '',
  'f1-score',
  '',
  '',
  'support\n\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '0',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.36',
  '',
  '',
  '',
  '',
  '',
  '0.28',
  '',
  '',
  '',
  '',
  '',
  '0.32',
  '',
  '',
  '',
  '',
  '',
  '',
  '417\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '1',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.57',
  '',
  '',
  '',
  '',
  '',
  '0.94',
  '',
  '',
  '',
  '',
  '',
  '0.71',
  '',
  '',
  '',
  '',
  '',
  '',
  '448\n',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '2',
  '',
  '',
  '',
  '',
  '',
  '',
  '0.88',
  '',
  '',
  '',
  '',
  '',
  '0.53',
  '',
  '',
  '',
  '',
  '',
  '0.66',
  '',
  '',
  '',
  '',
  '',
  '',
  '472\n\n',
  '',
  '',
  '',
  'accuracy',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

###### 

In [ ]:
print("Large-Size-Convolution-Maxpooling-model")
DIR = './Large-Size-Convolution-Maxpooling-model/'
modelpath = DIR + 'Large_CNN_Classifier.hdf5'
run_model(modelpath,testInSet, testOutSet)a  = {}